#  Inférence dudysfonctionnement  - Stratégie Multilabels
Dans ce Notebook, nous cosntruisons un modèle qui permet d'inférer DYSFONCTIONNEMENT à partir de la classification de l'incident et des données textuelles

Nous considérons ce problème comme un problème de classification multiclasses et multilabels. En effet, il y a plusieurs effets possibles et un incidents peut entrainer plusieurs effets.

Ainsi, notre métrique d'évaluation sera le f1_samples

Dans le Notebook précedent, nous n'avions pas pris en compte l'aspect multilabel et notre score était de  f1_weighted = 0,28.

Volontairement dans un premier temps, nous ne modifions pas les paramères de notre modèle afin d'avaluer l'apport de la stratégie multilabelles.

In [60]:
import warnings
warnings.filterwarnings('ignore')

from pprint import pprint
from time import time
import logging

import pandas as pd

import numpy as np
import sklearn as sk
import seaborn as sns

import nltk
from nltk import word_tokenize
lang ='french'

import clean_text
import skmultilearn




import matplotlib.pyplot as plt

from sklearn.ensemble import BaggingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit,KFold, ShuffleSplit
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import cross_val_score, cross_validate

from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score,f1_score,classification_report
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import TruncatedSVD,IncrementalPCA,SparsePCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.multiclass import OneVsRestClassifier

import spacy
nlp =spacy.load('fr')
from spacy.lang.fr.stop_words import STOP_WORDS

## 0.1 Chargement des données

In [20]:
%time
df_declaration_mrv = pd.read_csv("data/data_mrv/declaration_mrv_complet.csv")#delimiter=';',encoding='ISO-8859-1')
id_to_dco = pd.read_csv("data/ref_MRV/referentiel_dispositif.csv",delimiter=';',encoding='ISO-8859-1')
df_effets = pd.read_csv("data/ref_MRV/referentiel_dispositif_effets_connus.csv",delimiter=';',encoding='ISO-8859-1')
df_dys = pd.read_csv("data/ref_MRV/referentiel_dispositif_dysfonctionnement.csv",delimiter=';',encoding='ISO-8859-1')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 21.7 µs


In [63]:
df_dys

,DCO_ID,DCO,TDY_ID,TYPE_DYSFONCTIONNEMENT
0,2360,EXPLORATION FONCTIONNELLE ( DIVERS ),1854,NON FONCTIONNEMENT / FONCTIONNEMENT INCORRECT
1,2360,EXPLORATION FONCTIONNELLE ( DIVERS ),1859,NON RENSEIGNE
2,2360,EXPLORATION FONCTIONNELLE ( DIVERS ),1821,MESURE ERRONEE
3,3296,DMU D' APHERESE ECHANGE PLASMATIQUE,2012,RUPTURE D'INTEGRITE POCHE
4,3296,DMU D' APHERESE ECHANGE PLASMATIQUE,2015,RUPTURE D'INTEGRITE TUBULURE
...,...,...,...,...
18990,2641,SUBSTITUT OSSEUX ( ORTHOPEDIE ),1823,MIGRATION
18991,4981,RECHAUFFEUR DE SANG POUR MONITEUR D'HEMODIALYSE,1997,RUPTURE
18992,4981,RECHAUFFEUR DE SANG POUR MONITEUR D'HEMODIALYSE,1544,DECONNEXION
18993,4981,RECHAUFFEUR DE SANG POUR MONITEUR D'HEMODIALYSE,1854,NON FONCTIONNEMENT / FONCTIONNEMENT INCORRECT


## 0.2 Netoyage des données :
- Elimination de l'effet : PAS D'EFFET NEFASTE DECLARE
- Suppression des classes sous représentées (<15 occurences)
- Netoyyage des donénes textuelles
- Encodage de la classification et des effets

In [21]:
df_declaration_mrv['TDY_ID'].unique()

array(['D1790', 'D1859', 'D1848', 'D2091', 'D1689', 'D1932', 'D1886',
       'D1468', 'D1693', 'D1909', 'D1570', 'D1656', nan, 'D1997', 'D1538',
       'D1625', 'D1539', 'D1854', 'D2219', 'D2047', 'D1759', 'D1756',
       'D1852', 'D1936', 'D1627', 'D1856', 'D1646', 'D1951', 'D1464',
       'D2032', 'D1643', 'D1510', 'D1751', 'D1684', 'D2010', 'D1484',
       'D2216', 'D1713', 'D2026', 'D1985', 'D2033', 'D2006', 'D1755',
       'D1613', 'D1823', 'D1575', 'D1733', 'D2082', 'D2015', 'D1802',
       'D1675', 'D2012', 'D1712', 'D1628', 'D1870', 'D1813', 'D1730',
       'D1498', 'D1965', 'D1690', 'D1537', 'D1825', 'D1791', 'D1921',
       'D1917', 'D1663', 'D1913', 'D1864', 'D2039', 'D2256', 'D1795',
       'D1642', 'D1567', 'D1530', 'D1516', 'D1837', 'D1680', 'D1692',
       'D1658', 'D1891', 'D2009', 'D2042', 'D1954', 'D1682', 'D1905',
       'D1630', 'D1597', 'D1669', 'D1532', 'D2025', 'D2084', 'D2027',
       'D1486', 'D1520', 'D1919', 'D1535', 'D2023', 'D1569', 'D1652',
       'D2067',

In [53]:
%%time
#On complète les effets vide comme étant sans effets
df_declaration_mrv['TYPE_EFFET']  = df_declaration_mrv['TYPE_EFFET'].fillna("PAS D'EFFET NEFASTE DECLARE")
df_declaration_mrv['TEF_ID']= df_declaration_mrv['TEF_ID'].fillna('E1213')

#
df_declaration_mrv['TYPE_DYSFONCTIONNEMENT']  = df_declaration_mrv['TYPE_DYSFONCTIONNEMENT'].fillna("PAS DE DYSFONCTIONNEMENT")
df_declaration_mrv['TDY_ID']  = df_declaration_mrv['TDY_ID'].fillna("D0")

#on selectionne les colonnes avec des effets
df = df_declaration_mrv[['DESCRIPTION_INCIDENT','TYPE_VIGILANCE','LIBELLE_COMMERCIAL',
                         'REFERENCE_COMMERCIALE','ETAT_PATIENT','FABRICANT','DCO_ID',
                         'ACTION_PATIENT','CLASSIFICATION','TDY_ID']]#[df_declaration_mrv['TEF_ID']!='E1213']

# On complète les NaN avec du vide
df['ETAT_PATIENT'] = df['ETAT_PATIENT'].fillna("")
df['DESCRIPTION_INCIDENT'] = df['DESCRIPTION_INCIDENT'].fillna("")
df['LIBELLE_COMMERCIAL'] = df['LIBELLE_COMMERCIAL'].fillna("")
df['FABRICANT'] = df['FABRICANT'].fillna("")
df["REFERENCE_COMMERCIALE"] = df['REFERENCE_COMMERCIALE'].fillna("")
df['TYPE_VIGILANCE'] = df['TYPE_VIGILANCE'].fillna("")
df['CLASSIFICATION'] = df['CLASSIFICATION'].fillna('')
df['DCO_ID'] = df['DCO_ID'].fillna(-1)
#On nettoieles variables textueelles : 

for col in  ['DESCRIPTION_INCIDENT','LIBELLE_COMMERCIAL','ETAT_PATIENT','FABRICANT','ACTION_PATIENT'] :
    df[col] = df[col].map(lambda x: clean_text.preprocess_text(x))


print(len(df))
n = 15
# On filtre pour a voir plus de n observations par classse
df = df.groupby("TDY_ID").filter(lambda x: len(x) > n)
print(len(df))
#le_v = LabelEncoder()
#df.TYPE_VIGILANCE = le_v.fit_transform(df.TYPE_VIGILANCE.values)
le = LabelEncoder()
df.TDY_ID = le.fit_transform(df.TDY_ID.values)

df_m = df.groupby('DESCRIPTION_INCIDENT')['TDY_ID'].apply(list).reset_index(name='multilabels')


df_ = pd.merge(df,df_m, on = 'DESCRIPTION_INCIDENT')
df_['multilabels'] = df_['multilabels'].apply(np.array)
df_['multilabels'] = df_['multilabels'].map(np.unique)

#df_.to_csv('Multilabel_dataset.csv')

#df_ = df_.drop_duplicates('DESCRIPTION_INCIDENT')

76954
75248
CPU times: user 29.7 s, sys: 260 ms, total: 30 s
Wall time: 30 s


## 0.3 Construction du jeu de données d'entrainement et de test

In [54]:
# selection des train et test set
train_index,test_index = next(GroupShuffleSplit(random_state=1029).split(df_, groups=df_['DESCRIPTION_INCIDENT']))
df_train, df_test = df_.iloc[train_index], df_.iloc[test_index]
y = df_train.multilabels
y_test =df_test.multilabels

## 0.4 Encodage multilabel

In [55]:
lb = MultiLabelBinarizer()
y_lb =lb.fit_transform(y) 
y_test_lb = lb.transform(y_test)
X = df_train[['FABRICANT','CLASSIFICATION','DESCRIPTION_INCIDENT','TYPE_VIGILANCE','ETAT_PATIENT']]
X_test = df_test[['FABRICANT','CLASSIFICATION','DESCRIPTION_INCIDENT','TYPE_VIGILANCE','ETAT_PATIENT']]

## 1.1 Construction du pipeline avec une stratégie ONE-VS-REST

This strategy, also known as one-vs-all, is implemented in OneVsRestClassifier. The strategy consists in fitting one classifier per class. For each classifier, the class is fitted against all the other classes. In addition to its computational efficiency (only n_classes classifiers are needed), one advantage of this approach is its interpretability. Since each class is represented by one and only one classifier, it is possible to gain knowledge about the class by inspecting its corresponding classifier. This is the most commonly used strategy and is a fair default choice.

In [61]:
%%time
preprocess = ColumnTransformer(
    [('description_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 10000,norm = 'l2'), 'DESCRIPTION_INCIDENT'),
     
     ('etat_pat_tfidf', TfidfVectorizer(sublinear_tf=True, min_df=3,ngram_range=(1, 1),
                                       stop_words=STOP_WORDS,
                                       max_features = 10000,norm = 'l2'), 'ETAT_PATIENT'),
     
     ('fabricant_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 5000,norm = 'l2'), 'FABRICANT'),
    
    ('classification_enc', TfidfVectorizer(sublinear_tf=True, min_df=5,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 100,norm = 'l2'),'CLASSIFICATION'),
     
     ('type_vigilance_enc',OneHotEncoder(),'TYPE_VIGILANCE')
     ],
    
    remainder='passthrough')


pipeline = Pipeline([
    ('vect', preprocess),
    ('clf', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC(class_weight='balanced'),cv=3, method='isotonic'))),
])

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 339 µs


## 1.2 Evaluation du pipeline en cross validation : l'importance de la séparation train/test

In [ ]:
%%time
cv = KFold(n_splits=5, random_state=0)
result= cross_validate(pipeline, X, y_lb, scoring='f1_samples', cv=cv,n_jobs=-1)
result['test_score'].mean()

In [ ]:
%%time
cv = ShuffleSplit(n_splits=5, random_state=0)
result= cross_validate(pipeline, X, y_lb, scoring='f1_samples', cv=cv,n_jobs=-1)
result['test_score'].mean()

## 1.3 Si on supprimer les doublons ?

In [29]:
## Supression des doublons
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold


df_s = df_.drop_duplicates('DESCRIPTION_INCIDENT')

lb_s = MultiLabelBinarizer()
y_s = df_s.multilabels
y_s  = lb_s.fit_transform(y_s)

df_s = df_s[['FABRICANT','CLASSIFICATION','TYPE_VIGILANCE','DESCRIPTION_INCIDENT','ETAT_PATIENT']]

mskf = MultilabelStratifiedKFold(n_splits=2, random_state=0)

train_index_s,test_index_s = next(mskf.split(df_s,y_s))
df_train_s, df_test_s = df_s.iloc[train_index_s], df_s.iloc[test_index_s]
y_train_s = y_s[train_index_s]
y_test_s =y_s[test_index_s]

#lb_s = MultiLabelBinarizer()
#y_train_lb_s =lb_s.fit_transform(y_train_s) 
#y_test_lb_s = lb.transform(y_test_s)
X_train_s = df_train_s[['FABRICANT','CLASSIFICATION','TYPE_VIGILANCE','DESCRIPTION_INCIDENT','ETAT_PATIENT']]
X_test_s = df_test_s[['FABRICANT','CLASSIFICATION','TYPE_VIGILANCE','DESCRIPTION_INCIDENT','ETAT_PATIENT']]

pipeline = Pipeline([
    ('vect', preprocess),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight='balanced'))),
])

pipeline.fit(X_train_s,y_train_s)
#y_test_lb = lb.transform(y_test)
Y_pred_ovr_s = pipeline.predict(X_test_s)
f1 = f1_score(y_test_s , Y_pred_ovr_s,average='samples')
print('f1_score samples : ',f1)

f1_score samples :  0.46811272403864995


In [30]:
print(classification_report(y_test_s , Y_pred_ovr_s))

              precision    recall  f1-score   support

           0       0.72      0.74      0.73      3057
           1       0.00      0.00      0.00         9
           2       0.00      0.00      0.00         7
           3       1.00      0.10      0.18        10
           4       1.00      0.25      0.40         4
           5       0.67      0.40      0.50         5
           6       1.00      0.22      0.36         9
           7       0.64      0.26      0.37        27
           8       0.46      0.22      0.30        27
           9       0.50      0.14      0.22         7
          10       0.45      0.26      0.33        38
          11       0.50      0.07      0.12        14
          12       1.00      0.29      0.44         7
          13       0.41      0.36      0.38        36
          14       0.00      0.00      0.00        11
          15       0.45      0.16      0.23        57
          16       0.76      0.65      0.70        65
          17       0.50    

## 1.3 Evaluation du pipeline sur les données de test
### Avec le SVM probabilisé

In [62]:
%%time
pipeline.fit(X,y_lb)
#y_test_lb = lb.transform(y_test)
Y_pred_ovr = pipeline.predict(X_test)
f1 = f1_score(y_test_lb , Y_pred_ovr,average='samples')
print('f1_score samples : ',f1)

ValueError: 1D data passed to a transformer that expects 2D data. Try to specify the column selection as a list of one item instead of a scalar.

In [50]:
import joblib
filename = 'Encoder_dysfonctionnement.sav'
joblib.dump(le, filename)
filename = 'DYSFONCTIONNEMENT_model_0_55.sav'
joblib.dump(pipeline, filename)

['DYSFONCTIONNEMENT_model_0_55.sav']

In [32]:
print(classification_report(y_test_lb , Y_pred_ovr))

              precision    recall  f1-score   support

           0       0.87      0.83      0.85      5117
           1       0.00      0.00      0.00        16
           2       0.00      0.00      0.00         7
           3       1.00      0.25      0.40         4
           4       0.00      0.00      0.00         6
           5       0.00      0.00      0.00        14
           6       0.00      0.00      0.00         0
           7       0.60      0.17      0.26        18
           8       0.00      0.00      0.00        84
           9       0.00      0.00      0.00        12
          10       0.56      0.16      0.24        32
          11       0.00      0.00      0.00        15
          12       1.00      0.92      0.96        24
          13       0.14      0.33      0.19         9
          14       0.00      0.00      0.00         7
          15       0.69      0.21      0.32        43
          16       0.90      0.77      0.83        60
          17       0.00    

### Sans le SVM probabilisé

In [33]:
pipeline = Pipeline([
    ('vect', preprocess),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight='balanced'))),
])

In [34]:
%%time
pipeline.fit(X,y_lb)
Y_pred_ovr = pipeline.predict(X_test)
f1 = f1_score(y_test_lb , Y_pred_ovr,average='samples')
print('f1_score samples : ',f1)

f1_score samples :  0.5521267539039895
CPU times: user 2min 35s, sys: 380 ms, total: 2min 36s
Wall time: 2min 36s


In [35]:
print('f1_score samples : ',f1)

f1_score samples :  0.5521267539039895


In [36]:
print(classification_report(y_test_lb , Y_pred_ovr))

              precision    recall  f1-score   support

           0       0.87      0.83      0.85      5117
           1       0.00      0.00      0.00        16
           2       0.00      0.00      0.00         7
           3       1.00      0.25      0.40         4
           4       0.00      0.00      0.00         6
           5       0.00      0.00      0.00        14
           6       0.00      0.00      0.00         0
           7       0.60      0.17      0.26        18
           8       0.00      0.00      0.00        84
           9       0.00      0.00      0.00        12
          10       0.56      0.16      0.24        32
          11       0.00      0.00      0.00        15
          12       1.00      0.92      0.96        24
          13       0.14      0.33      0.19         9
          14       0.00      0.00      0.00         7
          15       0.69      0.21      0.32        43
          16       0.90      0.77      0.83        60
          17       0.00    

## Commentaire : 
#### **Les principaux enseignements sont les suivants :**
1)  L'approche multilabels est pertinente : Nous obtenons enfin un score "correct" 0.55 qui se rapproche de la précision à 4 du model simple Label. Ce qui est assez cohérent car il y a en moyenne 4 classes par evénement. Cela signifie que nous avons simplement tirer partie de la métrique mais que notre modèle n'apprend pas mieux... Dommage ! C'est en fait assez logique car le SVM en multiclasse applique déja une stratégie One vs REST

2)  Elle est pertinente.. Mais moins qu'on pourrait le penser avec la shuffle validation : comme le montre la différence entre le Kfold et le ShuffleSplit, les doublons joue un rôle important. Cette différence s'explique apr le fait que les doublons joue un rôle important dans notre base de données car ils ne sont pas exactement des "doublons". De plus ils nous permettent, de tenir compte de la proportion des classes dans notre evaluation.

3) Cette approche multilabels combinée avec la probalisation du SVM est couteuse en temps de construction de modèle et nous devrons le prendre en compte dans notre approche de finetuning.

#### **Les pistes d'améliorations sont :**

1) Finetuner le modèle pour vérifier que notre jeu de paramètre n'est pas un cas particulier de performances 


2) Essayer une approche
- MultiOutputClassifier
- ClassifierChain https://scikit-learn.org/stable/auto_examples/multioutput/plot_classifier_chain_yeast.html

3) Essayer le one-sht learning

## 2.1 Optune et la marge de progression du modèle
Pour le finetunning, nous enlevons la probabilisation du modèle

In [37]:
pipeline = Pipeline([
    ('vect', preprocess),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight='balanced'))),
])

In [38]:
import optuna
from optuna import Trial

def objective(trial):    
    
    train_index,test_index = next(GroupShuffleSplit(random_state=1029).split(X, groups=X['DESCRIPTION_INCIDENT']))
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    y_train_lb = lb.transform(y_train)
    y_test_lb = lb.transform(y_test)
    
    #Etat Patient
    #vect__etat_pat_tfidf__analyzer = trial.suggest_categorical('vect__etat_pat_tfidf__analyzer', ['word', 'char', 'char_wb']) 
    vect__etat_pat_tfidf__max_features = trial.suggest_int('vect__etat_pat_tfidf__max_features', 500, 20_000)
    vect__etat_pat_tfidf__min_df =  trial.suggest_int('vect__etat_pat_tfidf__min_df', 1,5)
    vect__etat_pat_tfidf__norm = trial.suggest_categorical('vect__etat_pat_tfidf__norm', ('l1', 'l2'))
    #Description
    #vect__description_tfidf__analyzer = trial.suggest_categorical('vect__description_tfidf__analyzer', ['word', 'char', 'char_wb']) 
    vect__description_tfidf__max_features = trial.suggest_int('vect__description_tfidf__max_features', 1500, 60_000)
    vect__description_tfidf__min_df =  trial.suggest_int('vect__description_tfidf__min_df', 1,5)
    vect__description_tfidf__norm = trial.suggest_categorical('vect__description_tfidf__norm', ('l1', 'l2'))
    #Fabricant
    vect__fabricant_tfidf__analyzer = trial.suggest_categorical('vect__fabricant_tfidf__analyzer', ['word', 'char', 'char_wb']) 
    vect__fabricant_tfidf__max_features = trial.suggest_int('vect__fabricant_tfidf__max_features', 500, 10_000)
    vect__fabricant_tfidf__min_df =  trial.suggest_int('vect__fabricant_tfidf__min_df', 1,5)
    vect__fabricant_tfidf__norm = trial.suggest_categorical('vect__fabricant_tfidf__norm', ('l1', 'l2'))
    #action patient
    
    #Classification
    vect__classification_enc__analyzer = trial.suggest_categorical('vect__classification_enc__analyzer', ['word', 'char', 'char_wb']) 
    vect__classification_enc__max_features = trial.suggest_int('vect__classification_enc__max_features', 500, 5000)
    vect__classification_enc__min_df =  trial.suggest_int('vect__classification_enc__min_df', 1,5)
    vect__classification_enc__norm = trial.suggest_categorical('vect__classification_enc__norm', ('l1', 'l2'))
    
    #clf__C =trial.suggest_loguniform('svr_c', 1e-5, 1e5)
    

    
    params = {
        #'vect__etat_pat_tfidf__analyzer':vect__etat_pat_tfidf__analyzer,
        'vect__etat_pat_tfidf__max_features': vect__etat_pat_tfidf__max_features,
        'vect__etat_pat_tfidf__min_df':vect__etat_pat_tfidf__min_df,
        'vect__etat_pat_tfidf__norm':vect__etat_pat_tfidf__norm,
        
        #'vect__description_tfidf__analyzer':vect__description_tfidf__analyzer,
        'vect__description_tfidf__max_features': vect__description_tfidf__max_features,
        'vect__description_tfidf__min_df':vect__description_tfidf__min_df,
        'vect__description_tfidf__norm':vect__description_tfidf__norm,
        
        'vect__fabricant_tfidf__analyzer':vect__fabricant_tfidf__analyzer,
        'vect__fabricant_tfidf__max_features': vect__fabricant_tfidf__max_features,
        'vect__fabricant_tfidf__min_df':vect__fabricant_tfidf__min_df,
        'vect__fabricant_tfidf__norm':vect__fabricant_tfidf__norm,
        
        
        'vect__classification_enc__analyzer':vect__classification_enc__analyzer,
        'vect__classification_enc__max_features': vect__classification_enc__max_features,
        'vect__classification_enc__min_df':vect__classification_enc__min_df,
        'vect__classification_enc__norm':vect__classification_enc__norm,
        
        #'clf__C':clf__C
    }
    
    pipeline.set_params(**params)
    pipeline.fit(X_train,y_train_lb)
    Y_pred_ovr = pipeline.predict(X_test)
    score = f1_score(y_test_lb , Y_pred_ovr,average='samples')
    
    return score

In [39]:
time_out = 75*100
studyName = 'EFFET_ml_optimisation_svm'
study = optuna.create_study(direction="maximize")
study.optimize(objective, timeout=time_out)

#Suvegarde du resultat
df = study.trials_dataframe()
df.to_json(studyName+'.json')
print(study.best_trial)

[I 2020-06-12 14:38:20,429] Finished trial#0 with value: 0.5502198450845048 with parameters: {'vect__etat_pat_tfidf__max_features': 2402, 'vect__etat_pat_tfidf__min_df': 2, 'vect__etat_pat_tfidf__norm': 'l2', 'vect__description_tfidf__max_features': 9720, 'vect__description_tfidf__min_df': 2, 'vect__description_tfidf__norm': 'l2', 'vect__fabricant_tfidf__analyzer': 'word', 'vect__fabricant_tfidf__max_features': 4471, 'vect__fabricant_tfidf__min_df': 1, 'vect__fabricant_tfidf__norm': 'l1', 'vect__classification_enc__analyzer': 'char_wb', 'vect__classification_enc__max_features': 1779, 'vect__classification_enc__min_df': 2, 'vect__classification_enc__norm': 'l2'}. Best is trial#0 with value: 0.5502198450845048.
[I 2020-06-12 14:49:14,466] Finished trial#1 with value: 0.5283789553711131 with parameters: {'vect__etat_pat_tfidf__max_features': 18618, 'vect__etat_pat_tfidf__min_df': 1, 'vect__etat_pat_tfidf__norm': 'l1', 'vect__description_tfidf__max_features': 47325, 'vect__description_tfid

KeyboardInterrupt: 

In [226]:
%%time
Param = {'vect__etat_pat_tfidf__max_features': 19700, 'vect__etat_pat_tfidf__min_df': 4, 'vect__etat_pat_tfidf__norm': 'l2', 
         'vect__description_tfidf__max_features': 20401, 'vect__description_tfidf__min_df': 2, 'vect__description_tfidf__norm': 'l2', 
         'vect__fabricant_tfidf__analyzer': 'word', 'vect__fabricant_tfidf__max_features': 9992, 'vect__fabricant_tfidf__min_df': 3, 
         'vect__fabricant_tfidf__norm': 'l1', 'vect__classification_enc_analyzer': 'word', 'vect__classification_enc__max_features': 2062,
         'vect__classification_enc__min_df': 4, 'vect__classification_enc__norm': 'l1'}

preprocess2 = ColumnTransformer(
    [('description_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=2,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 20401,norm = 'l2'), 'DESCRIPTION_INCIDENT'),
     
     ('etat_pat_tfidf', TfidfVectorizer(sublinear_tf=True, min_df=4,
                                        ngram_range=(1, 1),
                                        stop_words=STOP_WORDS,
                                        max_features = 19700,
                                        norm = 'l2'), 'ETAT_PATIENT'),
     
     ('fabricant_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 9992,norm = 'l1'), 'FABRICANT'),
    
    ('classification_enc', TfidfVectorizer(sublinear_tf=True, min_df=4,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 2062,norm = 'l2'),'CLASSIFICATION')
     ],
    
    remainder='passthrough')


pipeline = Pipeline([
    ('vect', preprocess2),
    ('clf', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC(class_weight='balanced'),cv=3, method='isotonic'))),
])


pipeline.fit(X,y_lb)
Y_pred_ovr = pipeline.predict(X_test)
f1 = f1_score(y_test_lb , Y_pred_ovr,average='samples')
print('f1_score samples : ',f1)

f1_score samples :  0.5491033504163741


In [227]:
print(classification_report(y_test_lb , Y_pred_ovr))

              precision    recall  f1-score   support

           0       0.36      0.20      0.26        49
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00        28
           3       0.78      0.31      0.44        68
           4       0.00      0.00      0.00        34
           5       0.45      0.37      0.41        68
           6       0.00      0.00      0.00       353
           7       1.00      0.76      0.86        21
           8       1.00      0.26      0.41        23
           9       0.00      0.00      0.00        34
          10       0.00      0.00      0.00        25
          11       0.00      0.00      0.00        22
          12       0.67      0.12      0.21        16
          13       0.00      0.00      0.00        11
          14       0.00      0.00      0.00        39
          15       0.00      0.00      0.00        18
          16       0.00      0.00      0.00         1
          17       0.00    

## Commantaire
 Notre première approximation des performances était la bonne. En effet, le finetunning des paramètres des TFIDF n'a pas permis d'augmenter significativmement les résultats
 
## 3.0 L'approche Multioutput

> Multioutput classification support can be added to any classifier with MultiOutputClassifier. This strategy consists of fitting one classifier per target. This allows multiple target variable classifications. The purpose of this class is to extend estimators to be able to estimate a series of target functions (f1,f2,f3…,fn) that are trained on a single X predictor matrix to predict a series of responses (y1,y2,y3…,yn).

https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html




In [40]:
from sklearn.multioutput import MultiOutputClassifier

pipeline = Pipeline([
    ('vect', preprocess),
    ('clf', MultiOutputClassifier(LinearSVC(class_weight='balanced'))),
])
#### prédiction 
pipeline.fit(X,y_lb)
Y_pred_ovr = pipeline.predict(X_test)
f1 = f1_score(y_test_lb , Y_pred_ovr,average='samples')
print('f1_score samples : ',f1)

f1_score samples :  0.5589696469390759


In [41]:
print(classification_report(y_test_lb , Y_pred_ovr))

              precision    recall  f1-score   support

           0       0.88      0.83      0.85      5117
           1       0.00      0.00      0.00        16
           2       0.00      0.00      0.00         7
           3       0.50      0.25      0.33         4
           4       0.00      0.00      0.00         6
           5       0.00      0.00      0.00        14
           6       0.00      0.00      0.00         0
           7       0.67      0.22      0.33        18
           8       0.40      0.10      0.15        84
           9       0.00      0.00      0.00        12
          10       0.60      0.19      0.29        32
          11       0.00      0.00      0.00        15
          12       1.00      0.75      0.86        24
          13       0.11      0.22      0.14         9
          14       0.00      0.00      0.00         7
          15       0.52      0.30      0.38        43
          16       0.90      0.73      0.81        60
          17       0.00    

### Commentaire
Comme attendu, nous n'observons pas de grande différence car les deux approches sont très similaires

## 3.1 Approche One vs One

>This strategy consists in fitting one classifier per class pair. At prediction time, the class which received the most votes is selected. Since it requires to fit n_classes * (n_classes - 1) / 2 classifiers, this method is usually slower than one-vs-the-rest, due to its O(n_classes^2) complexity. However, this method may be advantageous for algorithms such as kernel algorithms which don’t scale well with n_samples. This is because each individual learning problem only involves a small subset of the data whereas, with one-vs-the-rest, the complete dataset is used n_classes times.



In [42]:
%%time
from sklearn.multiclass import OneVsOneClassifier
pipeline = Pipeline([
    ('vect', preprocess),
    ('clf', MultiOutputClassifier(OneVsOneClassifier(LinearSVC(class_weight='balanced')))),
])
#### prédiction 
pipeline.fit(X,y_lb)
Y_pred_ovr = pipeline.predict(X_test)
f1 = f1_score(y_test_lb , Y_pred_ovr,average='samples')
print('f1_score samples : ',f1)

f1_score samples :  0.5589696469390759
CPU times: user 3min 28s, sys: 8.17 s, total: 3min 37s
Wall time: 3min 37s


### Commentaire
Nous n'oservons pas de changement de performances, seulement une hausse du temps de calcul
## 3.2 l'approche ClassifierChain
>A multi-label model that arranges binary classifiers into a chain.
Each model makes a prediction in the order specified by the chain using all of the available features provided to the model plus the predictions of models that are earlier in the chain.



In [43]:
from sklearn.multioutput import ClassifierChain

In [44]:
train_index_,test_index_ = next(GroupShuffleSplit(random_state=1029).split(X, groups=X['DESCRIPTION_INCIDENT']))
X_train_, X_test_ = X.iloc[train_index_], X.iloc[test_index_]
y_train_, y_test_ = y.iloc[train_index_], y.iloc[test_index_]

X_train_, X_test_ =preprocess.fit_transform(X_train_),preprocess.transform(X_test_)
y_train_lb_ = lb.transform(y_train_)
y_test_lb_ = lb.transform(y_test_)

In [45]:
%%time
X_train, X_test_ =preprocess.fit_transform(X),preprocess.transform(X_test)
clf = LinearSVC(class_weight='balanced')


chains = [ClassifierChain(clf, order='random', random_state=i) for i in range(10)]

for chain in chains:
    chain.fit(X_train, y_lb)
    
y_pred_chains = np.array([chain.predict(X_test_) for chain in chains])

chain_f1_scores = [f1_score(y_test_lb, y_pred_chain, average='samples') for y_pred_chain in y_pred_chains]

y_pred_ensemble = y_pred_chains.mean(axis=0)

y_e = y_pred_ensemble>=0.4

ensemble_f1_score = f1_score(y_test_lb,y_e, average='samples')

print(ensemble_f1_score)

0.5904407086021436
CPU times: user 34min 53s, sys: 2min 34s, total: 37min 27s
Wall time: 37min 27s


In [46]:
print(classification_report(y_test_lb , Y_pred_ovr))

              precision    recall  f1-score   support

           0       0.88      0.83      0.85      5117
           1       0.00      0.00      0.00        16
           2       0.00      0.00      0.00         7
           3       0.50      0.25      0.33         4
           4       0.00      0.00      0.00         6
           5       0.00      0.00      0.00        14
           6       0.00      0.00      0.00         0
           7       0.67      0.22      0.33        18
           8       0.40      0.10      0.15        84
           9       0.00      0.00      0.00        12
          10       0.60      0.19      0.29        32
          11       0.00      0.00      0.00        15
          12       1.00      0.75      0.86        24
          13       0.11      0.22      0.14         9
          14       0.00      0.00      0.00         7
          15       0.52      0.30      0.38        43
          16       0.90      0.73      0.81        60
          17       0.00    

In [ ]:
#Random test
test = np.random.rand(y_e.shape[0],y_e.shape[1])>=0.5
f1_score(y_test_lb_,test, average='samples')

### Commentaire : 
L'approche chain combiner à une méthode ensemblise améliore les performaces de quelques pourcent mais l'ordre de grandeur reste le même.

L'approche multilabel est une piste à suivre mais elle n'améliore pas ou peu la qualité de notre apprentissage.

In [247]:
C = sk.metrics.multilabel_confusion_matrix(y_test_lb,y_e)

In [257]:
df_re = pd.DataFrame(classification_report(y_test_lb , Y_pred_ovr,output_dict=True)).T

In [273]:
df_re['Class'] = le.inverse_transform(df_re.index.map(int).values[:-5])

ValueError: invalid literal for int() with base 10: 'micro avg'

In [269]:
L =df_re.index.to_list()

In [271]:
L.map(int)

AttributeError: 'list' object has no attribute 'map'